### 人工智慧期末專題

主題：Sentiment140 dataset with 1.6 million tweets

資料來源：https://www.kaggle.com/datasets/kazanova/sentiment140/data

In [1]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string, nltk, re
from gensim.models import Word2Vec
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
import tensorflow_hub as hub
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
drive.mount('/content/drive', force_remount = True)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
df = pd.read_csv('/content/drive/MyDrive/1122/人工智慧/project/training.1600000.processed.noemoticon.csv', encoding = 'latin-1', header = None)
df = df[[5, 0]]
df.columns=['tweet', 'Sentiment']
df.loc[df["Sentiment"] == 4, "Sentiment"] = 1

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   tweet      1600000 non-null  object
 1   Sentiment  1600000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [7]:
positive = df[df['Sentiment'] == 0].sample(n = 8000, random_state = 0)
negative = df[df['Sentiment'] == 1].sample(n = 2000, random_state = 0)
data = pd.concat([positive,negative])
data = data.sample(frac = 1, random_state = 0).reset_index(drop = True)

In [ ]:
data = df

In [8]:
def preprocess(text):
  text = text.lower()
  text = ''.join([word for word in text if word not in string.punctuation])
  tokens = word_tokenize(text)
  tokens = [word for word in tokens if word not in stop_words]
  lemmatizer = WordNetLemmatizer()
  lemma_words = [lemmatizer.lemmatize(w, pos = 'a') for w in tokens]
  return ' '.join(lemma_words)

data["tweet"] = data["tweet"].apply(preprocess)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet      10000 non-null  object
 1   Sentiment  10000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 156.4+ KB


### Word2Vec ( Using Gensim Package )

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data["tweet"], data["Sentiment"], test_size = 0.3, random_state = 0)
sentences = [sentence.split() for sentence in X_train]
w2v_model = Word2Vec(sentences, vector_size = 100, window = 5, min_count = 5, workers = 4)

In [ ]:
def vectorize(sentence):
  words = sentence.split()
  words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
  if len(words_vecs) == 0:
    return np.zeros(100)
  words_vecs = np.array(words_vecs)
  return words_vecs.mean(axis = 0)

X_train = np.array([vectorize(sentence) for sentence in X_train])
X_test = np.array([vectorize(sentence) for sentence in X_test])

In [ ]:
clf = LogisticRegression()
history = clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.76      0.74      0.75    239877
           1       0.75      0.77      0.76    240123

    accuracy                           0.75    480000
   macro avg       0.75      0.75      0.75    480000
weighted avg       0.75      0.75      0.75    480000



In [ ]:
# old
clf = LogisticRegression()
history = clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.80      1.00      0.89      4779
           1       0.00      0.00      0.00      1221

    accuracy                           0.80      6000
   macro avg       0.40      0.50      0.44      6000
weighted avg       0.63      0.80      0.71      6000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Bert ( Using Transformers )

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 1)
encoded_data = tokenizer(data['tweet'].tolist(), padding = True, truncation = True, return_tensors = 'tf')
input_ids = np.array(encoded_data['input_ids'])
labels = np.array(data['Sentiment'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(input_ids, labels, test_size = 0.3, random_state = 0)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.5, random_state = 0)
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)
valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid)).batch(32)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits = True)
model.compile(optimizer = optimizer, loss = loss, metrics = ['accuracy'])
history = model.fit(train_dataset, epochs = 1, validation_data = valid_dataset)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
 58/110 [==============>...............] - ETA: 21:02 - loss: 0.5096 - accuracy: 0.8033

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits = True)
model.compile(optimizer = optimizer, loss = loss, metrics = ['accuracy'])
history = model.fit(train_dataset, epochs = 1, validation_data = valid_dataset)
prediction = model.predict(test_dataset)
print(classification_report(y_test, prediction))

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
219/219 [==============================] - 14164s 64s/step - loss: 0.5047 - accuracy: 0.8023 - val_loss: 0.5021 - val_accuracy: 0.7965


In [7]:
# old
optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits = True)
model.compile(optimizer = optimizer, loss = loss, metrics = ['accuracy'])
model.fit(train_dataset, epochs = 1, validation_data = valid_dataset)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
219/219 [==============================] - 14313s 65s/step - loss: 0.5029 - accuracy: 0.8023 - val_loss: 0.4946 - val_accuracy: 0.7965


In [ ]:
# model.evaluate(test_dataset)

219/219 [==============================] - 4017s 18s/step - loss: 0.4967 - accuracy: 0.8007


[0.4966564476490021, 0.8007143139839172]

In [ ]:
prediction = model.predict(test_dataset)
print(len(prediction))

In [8]:
prediction = model.predict(test_dataset)
print(classification_report(y_test, prediction))

219/219 [==============================] - 3393s 15s/step


ValueError: Found input variables with inconsistent numbers of samples: [7000, 1]